**Objectives of this Webinar:**

You will be able to query SHARE for records that are known to be tied to your institution, and might be tied to your institution.  You will then be able to export them to a csv file for review.

Let's start in the Search Interface: http://share.osf.io

**Replicating this process in code**

First we will set some values for Jupyter to use.

Why is this useful?

Remember Drew Barrymore in '50 First Dates'?

http://www.imdb.com/title/tt0343660/

**Special Tip:**  Put your cursor in the Cell (i.e., box) below and hit Shift+Enter on your keyboard to run it 

In [ ]:
SHARE_API = 'https://staging-share.osf.io/api/search/abstractcreativework/_search'

This is now in memory!  However, beware.  Just like Drew Barrymore if you close this notebook and reopen it, jupyter has completely forgotten about it, and you will need to rerun it for Jupyter to know about it.

**Special Tip:** Why all CAPS for 'SHARE_API'? It is convention (i.e., common practice) to use all capital letters when defining something that does not change?

Next, let's move onto to an example simple query to SHARE.

In [ ]:
import furl
import requests

search_url = furl.furl(SHARE_API)
search_url.args['size'] = 3
recent_results = requests.get(search_url.url).json()

recent_results = recent_results['hits']['hits']

print('The request URL is {}'.format(search_url.url))
print('----------')
for result in recent_results:
    print(
        '{} -- from {}'.format(
            result['_source']['title'],
            result['_source']['sources']
        )
    )

Let's add a search string

**Special Tip:** Sometimes you can learn a lot from a url and let's look at it after we search using the search box on SHARE (http://share.osf.io)
    
https://share.osf.io/discover?q=university%20of%20oregon

In [ ]:
search_url.args['q'] = 'university of oregon'
recent_results = requests.get(search_url.url).json()

recent_results = recent_results['hits']['hits']

print('The request URL is {}'.format(search_url.url))
print('---------')
for result in recent_results:
    print(
        '{} -- from {}'.format(
            result['_source']['title'],
            result['_source']['sources']
        )
    )

**Special Tip:**  To print text and variables together use the format function as below with '{}' anywhere you want to insert a value and then include them in order to the format method.

**Looking for exact matches use our facet filter**

Let's go back to SHARE, apply a facet and see what it is doing

http://share.osf.io
    

In [ ]:
affiliation_query = {
    "size": 10000,
    "query": {
        "bool": {
            "must": {
                "query_string": {
                    "query": "*"
                }
            },
            "filter": [
                {
                    "term": {
                        "institutions.raw": "University of Oregon Libraries"
                    }
                }
            ]
        }
    }
}

Let's input a method into memory so we can use it later.  Do not worry too much about the specifics yet.

**Special Tip:** Just like variables you can define methods that do a job and you can then use them later without rewriting what it does

In [ ]:
import json
import requests

def query_share(url, query):
    # A helper function that will use the requests library,
    # pass along the correct headers,
    # and make the query we want
    headers = {'Content-Type': 'application/json'}
    data = json.dumps(query)
    return requests.post(url, headers=headers, data=data, verify=False).json()

**Special Tip:** Use '#' as first character in a line if you want to put in a comment

In [ ]:
affiliation_results = query_share(search_url.url, affiliation_query)

total_results = affiliation_results['hits']['total']
print('total results found: {}'.format(total_results))

Where are my results?  Things can differently from staging to production environments.

Let's create a new variable and change our search url

In [ ]:
PROD_SHARE_API= 'https://share.osf.io/api/search/abstractcreativework/_search'

search_url = furl.furl(PROD_SHARE_API)
search_url.args['size'] = 3

In [ ]:
affiliation_results = query_share(search_url.url, affiliation_query)

total_results = affiliation_results['hits']['total']
print('total results found: {}'.format(total_results))

Next let's dive into the results to see what is being returned and how to then push it into a csv file

**Special Tip:** The built-in 'print' is a very easy way to explore what is happening

In [ ]:
affiliation_results = query_share(search_url.url, affiliation_query)
print('------------')
print(affiliation_results)
print('------------')

This is a lot of stuff and we need to dive in fartheR

**Special Tip**: SHARE returns this in what is called a Dictionary in Python (i.e. map, hash) and you supply the name of 'key' to then get the 'value'

**Special Tip**: You can also get the list of keys for a dictionary by calling the dict method.

So, we will print out our list of keys

In [ ]:
print(affiliation_results.keys())

It looks like what we need is 'hits'

**Special Tip:** Make sure you put '' (single quotes) around the key name

In [ ]:
affiliation_results = query_share(search_url.url, affiliation_query)
print(affiliation_results['hits'])

This is better but we are still not quite there.  Let's call keys again one level down

In [ ]:
print(affiliation_results['hits'].keys())

OK, it looks like we need to add another hits

**Special Tip:** You can access multiple levels of nested dictionaries by adding additional '[]' to the variable

In [ ]:
print(affiliation_results['hits']['hits'])

Let's check our next set of nested keys...

In [ ]:
print(affiliation_results['hits']['hits'].keys())

Did you get an error?

What just happened here?  We have now encountered 'list' and that does not have a notion of keys.  List is essentially just a pile of records that has an order (first, second,...,last) but no keys assigned.  So, how do we interact with that?

Let's take a look at the documentation by doing a google search for python list...

**Interacting with lists**

It looks like maybe we have an item embedded in a list, and this list is actually contains an item per record returned by our search.

What we really want though for outputting to csv is the field names to use as column headers

So, let's see if we can get to that based on the 'pop' method we found in the documentation here: https://docs.python.org/3/tutorial/datastructures.html

In [ ]:
print(affiliation_results['hits']['hits'].pop())

Close again.  It looks like what we need is the key '_source', and then maybe we can get our column headers...

In [ ]:
record = affiliation_results['hits']['hits'].pop()
print(record['_source'].keys())

We are REALLY close now, but not quite there.  Let's keeping digging and add one additional thing, a for loop

**Special Tip:** You can move through a list of elements within a dictionary or list with a for loop.  

In [ ]:
record = affiliation_results['hits']['hits'].pop()

for key in record['_source'].keys(): 
    print(key)

What happened here?

Why is the list empty?  When you pop something off the list, it is gone once you leave a 'cell'.  So we need to rerun our search and recreate our list.

In [ ]:
affiliation_results = query_share(search_url.url, affiliation_query)
record = affiliation_results['hits']['hits'].pop()
for key in record['_source'].keys(): 
    print(key)

**Success!**    We have it, our list of fields and column headers!

**Special Tip:** An advanced move would be to create a method that gives you your list of keys

Now let's look at a record and print it to the output...

In [ ]:
#put keys and values together
for key in record['_source'].keys():
    print('{}:{}'.format(key,record['_source'][key]))

Let's further clean this up a little bit to make it easier to see what is happening...

**Special Tip:** By breaking steps apart it makes code much easier to understand for everyone

In this case we will add a new variable

In [ ]:
#put keys and values together
record_source = record['_source']

for key in record_source.keys():
    print('{}:{}'.format(key,record_source[key]))

**Writing output to CSV**

We are now ready to move to CSV.

**Special Tip:** Copy, Paste, and Adapt...It is always easier to start from an example.

Let's try writing to a simple csv file as an example (example pulled from https://docs.python.org/3/library/csv.html)

**Special Tip:** Add print statements to show something started and finished when there is no output

In [ ]:
import csv

print('---begin writing file---')

with open('names.csv', 'w') as csvfile:
    fieldnames = ['first_name', 'last_name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow({'first_name': 'Baked', 'last_name': 'Beans'})
    writer.writerow({'first_name': 'Lovely', 'last_name': 'Spam'})
    writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})
    
print('---done writing file----')

Did you find the file?  It should put it in whatever is the home directory for Jupyter.  This is likely your user folder.

Let's adapt this now with our data from SHARE...

In [ ]:
import csv

affiliation_results = query_share(search_url.url, affiliation_query)
records = affiliation_results['hits']['hits']

print('---begin writing file---')

#set our filenames
SHARE_MATCHING_INSTITUTION_RECORDS = 'share_matching.csv'

with open(SHARE_MATCHING_INSTITUTION_RECORDS, 'w') as csvfile:
    #instead of pop use a for loop
    i = 0
    for record in records:
        # we need to write out the header
        record_source = record['_source']
        if i == 0:
            fieldnames = record_source.keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            i = i + 1
        writer.writerow(record_source)    
        
print('---done writing file----')

What happened here?  

It was confused by a nested dictionary and fieldnames it did not expect.

**Special Tip:** 'extrasaction' value of 'ignore' will not throw an error if there are keys in this method such as a nested dictionary

In [ ]:
import csv

affiliation_results = query_share(search_url.url, affiliation_query)
records = affiliation_results['hits']['hits']

print('---begin writing file---')

#set our filenames
SHARE_MATCHING_INSTITUTION_RECORDS = 'share_matching.csv'

with open(SHARE_MATCHING_INSTITUTION_RECORDS, 'w') as csvfile:
    #instead of pop us a for loop
    i = 0
    for row in records:
        if i == 0:
            fieldnames = row['_source'].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction='ignore')
            writer.writeheader()
            i = i + 1
        writer.writerow(row['_source'])    
        
print('---done writing file---')

Let's get all the records as this limit is set to 10 by default

**Special Tip:** There is a parameter to elastic search that we querying where the default value is 10.  You can set this to a larger value, but the maximum is 10000

Let's add this to our query and then recreate our csv file

In [ ]:
affiliation_query = {
    "size": 10000,
    "query": {
        "bool": {
            "must": {
                "query_string": {
                    "query": "*"
                }
            },
            "filter": [
                {
                    "term": {
                        "institutions.raw": "University of Iowa"
                    }
                }
            ]
        }
    }
}

In [ ]:
import csv

affiliation_results = query_share(search_url.url, affiliation_query)
records = affiliation_results['hits']['hits']

print('---begin writing file---')

#set our filenames
SHARE_MATCHING_INSTITUTION_RECORDS = 'share_matching.csv'

with open(SHARE_MATCHING_INSTITUTION_RECORDS, 'w') as csvfile:
    #instead of pop us a for loop
    i = 0
    for row in records:
        if i == 0:
            fieldnames = row['_source'].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction='ignore')
            writer.writeheader()
            i = i + 1
        writer.writerow(row['_source'])    
        
print('---done writing file---')

**Wrapping Up**

You have now successfully queried share for records tied to your institution and have pushed them to a csv file.

**Future Steps**

-There is more cleanup to be done to get other values like if something has more than one link by adding nested keys to the field names list as well will capture these values

-Once you have the data, update and then push it back to SHARE

**Other tips...**

You can use the os module as shown below to change directories to a certain path if you want to put the csv file in a particular directory.

In [ ]:
import os

#os.getcwd()
#os.chdir('..')
#os.getcwd()
#os.chdir('share_tutorials')
#os.getcwd()